In [ ]:
from glob import glob
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import xmitgcm as xmit
import convertMITgcmTime.convertMITgcmTime as cmt
import xgcm
import pickle
import eddytools as et
from cmocean import cm

import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)
# Obviously it is not a great idea to ignore warnings, however there are quite many
# RuntimeWarnings because of division by 0 in some parts of this notebook. To keep
# the instructive nature of this example notebook, these warnings are ignored.

In [ ]:
#from dask.distributed import Client, LocalCluster

In [ ]:
#cluster = LocalCluster(n_workers=3, threads_per_worker=2)
#client = Client(cluster)
#client

In [ ]:
# paths
datapath = '/home/jan/Data/MITgcm_test_data/'
gridpath = '/home/jan/Data/MITgcm_test_data/grid/'

### Loading data
In case the MITgcm output is in binary format (`*.meta` and `*.data`), use `xmitgcm` to load the data. If the data is in `netCDF` format, make sure that you define all the necessary grid information for `xgcm` to work properly! Because the time dimension of the MITgcm model calendar does not really contain information about the calendar used, we create a new time dimension that can be understood by `xarray` with [`convertMITgcmTime`](https://github.com/jk-rieck/convertMITgcmTime).

In [ ]:
# load data
start = 1555200
end = 1580760
step = 360
dt = 1200.
data = xmit.open_mdsdataset(datapath, iters=list(np.arange(start, end + 1, step)), delta_t=dt,
                            ignore_unknown_vars=True, grid_dir=gridpath, geometry="cartesian", calendar="model", prefix="3D_diags_5d")
data = data.chunk({'time': 34, 'Z': 55, 'Zl': 55, 'Zu': 55, 'Zp1': 56, 'XC': 250, 'XG': 250, 'YC': 300, 'YG': 300})
data["time"] = cmt.convert2cftime360(data.time.dt.days.values)

In [ ]:
# define metrics for xgcm (not strictly necessary)
data['drW'] = data.hFacW * data.drF #vertical cell size at u point
data['drS'] = data.hFacS * data.drF #vertical cell size at v point
data['drC'] = data.hFacC * data.drF #vertical cell size at tracer point
data = data.set_coords(["drW", "drS", "drC"])

metrics = {
    ('X'): ['dxC', 'dxG', 'dxF', 'dxV'], # X distances
    ('Y'): ['dyC', 'dyG', 'dyF', 'dyU'], # Y distances
    ('Z'): ['drF', 'drW', 'drS', 'drC'], # Z distances
    ('X', 'Y'): ['rAw', 'rAs', 'rA', 'rAz'] # Areas in x-y plane
}

In [ ]:
data

### OKUBO-WEISS
We use xgcm to define the grid for the interpolation and differentiation necessary for the calculation of vorticity and the Okubo-Weiss parameter.  
For more information on the Okubo-Weiss parameter and its calculation see [Okubo, 1970](https://doi.org/10.1016/0011-7471(70)90059-8); [Weiss, 1991](https://doi.org/10.1016/0167-2789(91)90088-Q); [Basdevant and Philipovitch, 1994](https://doi.org/10.1016/0167-2789(94)90222-4); [Chelton et al., 2007](https://doi.org/10.1029/2007GL030812).

In [ ]:
grid = xgcm.Grid(data, metrics=metrics)

In [ ]:
# Calculate vorticity and Okubo-Weiss parameter
data_OW = et.okuboweiss.calc(data, grid, 'UVEL', 'VVEL')

In [ ]:
# Merge the new variables `OW` and `vort` to the dataset `data` and make sure the chunk sizes are as before.
data = xr.merge([data, data_OW], compat='override').chunk({'XC': 250, 'XG': 250, 'YC': 300, 'YG': 300})

### INTERPOLATION
Now we interpolate the data. If the data is entirely on a grid, that has one-dimensional latitude and longitude dimensions (i.e. longitude at one x-index does not change with changing y-index and latitude at one y-index does not change with changing x-index), the fields are simply interpolated to the F-point of the grid (vorticity and OW paramter are calculated and stored already on the F-point).
If the grid is curivlinear, e.g. North of 20S or so on the ORCA grids, a regridding is performed.  
Note that the interpolation can take VERY long for large region and/or long time periods. If one has to deal with problems regarding the time it takes, one could split the interpolation into several parts, let's say interpolate one year at a time, store the results and later load the combined interpolated data.

The longitude in all the parameter definitions (`lon1` and `lon2`) needs to be in the range (-180, 180) for `latlon` grids. Longitudes larger then 180 will result in an error. For cartesian grids, longitudes must be positive with meter as units.

IMPORTANT:  
The region that gets interpolated needs to be larger than the region used later for detection and tracking. This is required to ensure that eddies close to the boundaries of the detection region can be detected, tracked and sampled, even if their surroundings extend beyond the limits of the detection region.  
To handle this, the region is internally extended by 2 degrees (for a `latlon` grid) or 200 km (for a cartesian grid) in each longitude direction and 1 degree (`latlon`) or 100km (cartesian) in latitude direction. So you need to make sure that the region you specify is not closer to the limits of your data that that.  
If you want to detect eddies in the region 140-170E and 50-60S, the interpolation will be carried out within 138-172E and 49-61S! However, you still need to define 140-170E and 50-60S in the `interpolation_parameters` (or something equivalent for the cartesian grids).  

Before the interpolation, we need to define a ocean mask at the F-point, as the MITgcm does not write this as output by default. For that we interpolate the ocean mask at the V point (`maskS`) in X direction, to shift it onto the F-point, then we decide that the mask should define ocean everywhere, where it is not zero. I.e. if an ocean point (`1`) and a land point (`0`) are interpolated (=`0.5`), this is converted to ocean (`1`).

In [ ]:
# add maskZ for interpolation (mask at F point)
data['maskZ'] = grid.interp(data['maskS'], "X", to="left", metric_weighted=["X", "Y"])
data['maskZ'] = data['maskZ'].where(data['maskZ']==1, other=0)
data = data.set_coords(["maskZ"]).chunk({'XC': 250, 'XG': 250, 'YC': 300, 'YG': 300})

If you are not 100% sure yet how big the region will be in which you want to detect the eddies, you can specify a larger region for the interpolation and then later define smaller regions for the detection, tracking, etc.

In [ ]:
# Define the parameters for the interpolation
interpolation_parameters = {'model': 'MITgcm',
                            'grid': 'cartesian',
                            'start_time': '0061-01-01', # time range start
                            'end_time': '0061-12-30', # time range end
                            'calendar': '360_day', # calendar, must be either 360_day or standard
                            'lon1': 0.6e6, # minimum longitude of detection region
                            'lon2': 1.3e6, # maximum longitude
                            'lat1': 1.0e6, # minimum latitude
                            'lat2': 1.3e6, # maximum latitude
                            'res': 10., # resolution of the fields in km
                            'vars_to_interpolate': ['OW', 'vort', 'THETA'], # variables to be interpolated 
                            'mask_to_interpolate': ['maskZ', 'maskC', 'Depth']} # masks to interpolate

In [ ]:
data_int = et.interp.horizontal(data, metrics, interpolation_parameters)

Don't be confused that interpolation of `dxV` and `dyU` is done although we did not request that. `maskZ`, `dxV` and `dyU` are always interpolated, even if you do not request that because they are essential for the detection of the eddies!

### Threshold for the Okubo-Weiss parameter
Some sort of threshold needs to be defined for the Okubo-Weiss parameter to distinguish regions within and outside of eddies. This is usually defined as `OW_thr = a * sigma` ([Chelton et al., 2007](https://doi.org/10.1029/2007GL030812)), where `sigma` is the spatial standard deviation of the Okubo-Weiss parameter and `a` is some factor (more on the factor later).  
In regions with highly variable dynamic conditions, detection based on a single values of this standard deviation often fails to detect eddies in more "quiet" areas of the ocean ([Isern-Fontanet et al., 2003](https://doi.org/10.1175/1520-0426(2003)20<772:IOMEFA>2.0.CO;2)).Thus we here define a spatially variable threshold for the Okubo-Weiss parameter by calculating the (2D) spatial standard deviation in moving boxes.

In [ ]:
# Load `OW` into memory so the `.rolling` operation is faster
OW_tmp = data_int['OW'].sel(z=100, method="nearest").compute()

In [ ]:
 # Convert all land values to `NaN` so we don't have a lot of zeros when calculating
# the standard deviation
OW_tmp = OW_tmp.where(OW_tmp != 0)
lon_tmp = OW_tmp['lon'].where(OW_tmp['lon'] > 0, other=OW_tmp['lon'] + 360.)
OW_tmp = OW_tmp.assign_coords({'lon': lon_tmp})

In [ ]:
mean_OW_spatial_std = OW_tmp.rolling(
                          lon=100, center=True, min_periods=1
                          ).std(skipna=True).rolling(
                          lat=100, center=True, min_periods=1
                          ).std(skipna=True).mean('time')

It is also possible to use the mean spatial standard deviation of the Okubo-Weiss parameter over the whole considered region. To do that execute the cell below instead of the one above! Please note that `'OW_thr'` in `detection_parameters` needs to be defined differently, depending on which method of calculating the standard deviation you choose.

In [ ]:
#mean_OW_spatial_std = OW_tmp.std(('lon', 'lat'), skipna=True).mean('time') # uncomment if necessary

In [ ]:
# use if OW_std is 2D
data_int = data_int.update({'OW_std': (['lat', 'lon'], mean_OW_spatial_std.values)})

In [ ]:
# use if OW_std is 1D
#data_int = data_int.update({'OW_std': mean_OW_spatial_std})

In [ ]:
# Store the interpolated field on disk so we do not have to do the interpolation again
data_int.to_netcdf(datapath + 'test_data.nc', mode='w')

In [ ]:
# If we already did the interpolation earlier and now start off with the interpolated fields,
# we just uncomment the line below to load them
data_int = xr.open_dataset(datapath + 'test_data.nc')

In [ ]:
# Let's have a look at the fields
plt.pcolormesh(data_int['OW'].isel(time=40, z=20), vmin=-1e-10, vmax=1e-10, cmap=cm.balance)
plt.colorbar()

In [ ]:
plt.pcolormesh(data_int['OW_std'], vmin=-1e-10, vmax=1e-10, cmap=cm.balance)
plt.colorbar()

We see several eddy-like features that have a low OW-parameter, now let's see whether the algorithm detects them as eddies!

### DETECTION

Things to note for the setting of `detection_parameters`:  
1. `'lon1'` and `'lon2'` need to be positive and in meters. 
2. `'OW_thr': data_int` if the field `OW_std` in `data_int` is 2D, i.e. we have a spatially varying standard deviation of the Okubo-Weiss parameter. `'OW_thr': data_int['OW_std'].values` if the field `OW_std` in `data_int` is 1D.  
3. `'OW_thr_factor'` is usually chosen in the range `(-0.5, -0.2)` ([Chelton et al., 2007](https://doi.org/10.1029/2007GL030812); [Isern-Fontanet et al., 2003](https://doi.org/10.1175/1520-0426(2003)20<772:IOMEFA>2.0.CO;2)).

In [ ]:
# Specify parameters for eddy detection
detection_parameters = {'model': 'MITgcm',
                        'grid': 'cartesian',
                        'start_time': '0061-01-01', # time range start
                        'end_time': '0061-12-30', # time range end
                        'calendar': '360_day', # calendar, must be either 360_day or standard
                        'lon1': 0.6e6, # minimum longitude of detection region
                        'lon2': 1.3e6, # maximum longitude
                        'lat1': 1.0e6, # minimum latitude
                        'lat2': 1.3e6, # maximum latitude
                        'res': 10., # resolution of the fields in km
                        'min_dep': 1000, # minimum ocean depth where to look for eddies in m
                        'OW_thr': data_int, # 
                        'OW_thr_name': 'OW_std', # Okubo-Weiss threshold for eddy detection
                        'OW_thr_factor': -0.3, # Okubo-Weiss parameter threshold
                        'Npix_min': 20, # minimum number of pixels (grid cells) to be considered as eddy
                        'Npix_max': 500} # maximum number of pixels (grid cells)

In [ ]:
# We detect eddies at a depth of 100 m
eddies = et.detection.detect_OW(data_int.sel(z=100, method='nearest'), detection_parameters, 'OW', 'vort')

In [ ]:
# In this format, information for every detected eddy is stored
# (eddies[t][ed], where t is the time step and ed the eddy number)
eddies[0][1]

In [ ]:
# Plot a single time step to see what the detection method detected
t=40
d_p = detection_parameters
plot_lon = data_int['lon'].where(data_int['lon'].values > 0, other=data_int['lon'].values + 360)
plt.pcolormesh(plot_lon, data_int['lat'].values, data_int.OW.isel(time=t).sel(z=100, method='nearest').values,
               vmin=-1e-10, vmax=1e-10, cmap=cm.balance)
for c, l in zip(['whitesmoke', 'dimgray'], ['-', '--']):
    plt.plot([d_p['lon1'], d_p['lon2'], d_p['lon2'], d_p['lon1'], d_p['lon1']],
             [d_p['lat1'], d_p['lat1'], d_p['lat2'], d_p['lat2'], d_p['lat1']],
             color=c, linestyle=l)

for i in np.arange(0, len(eddies[t])-1):
    if eddies[t][i]['lon'] < 0:
        eddy_lon = eddies[t][i]['lon'] + 360
    else:
        eddy_lon = eddies[t][i]['lon']
    for s, c in zip([10, 8], ['k', 'gold']):
        plt.plot(eddy_lon, eddies[t][i]['lat'], marker='o', color=c, markersize=s)

We see that all features below a certain OW threshold have been detected. Some of them, we might not consider as an eddy. If you have the feeling, that two many features are detected, that are not eddies, change parameters like `OW_thr_factor`, `Npix_min`, `Npix_max`

In [ ]:
# For every time step, we store one file on disk with all the information of the detected
# eddies at this time step
for i in np.arange(0, len(eddies)):
    datestring = str(eddies[i][0]['time'])[0:10]
    with open(datapath + 'test_'
          + str(datestring) + '_eddies_OW0.3_test.pickle', 'wb') as f:
        pickle.dump(eddies[i], f, pickle.HIGHEST_PROTOCOL)
    f.close()

### TRACKING

Things to note for the setting of `tracking_parameters`:  
1. `'start_time'` is required to be no earlier than the earliest actual date of the detected eddies. In our case here, for the year 0002 and a 5-day temporal resolution of the data, this is `'0002-01-05'` (The `MITgcm` stores the 5-daily averages at the end of the 5-day period).
2. `'lon1'` and `'lon2'` need to be identical to `'lon1'` and `'lon2'` in `detection_parameters`.  
3. If you stored the detected eddies in files and want to track these, set `'dict': 0` and make sure `'data_path'`, `'file_root'` and `'file_spec'` are set accordingly. The method will look for files `datapath + file_root + 'YYYYMMDD' + file_spec + '.pickle'`, the date `'YYYYMMDD'` is automatically calculated from `'start_time'`, `'dt'`, and `'end_time'`. You have to make sure that the stored, detected eddies contain that date in their filename (e.g. as defined in the cell above)!

In [ ]:
# Specify parameters for eddy tracking
tracking_parameters = {'model': 'MITgcm',
                       'grid': 'cartesian',
                       'start_time': '0061-01-03', # time range start
                       'end_time': '0061-12-30', # time range end
                       'calendar': '360_day', # calendar, must be either 360_day or standard
                       'dt': 5, # temporal resolution of the data in days
                       'lon1': 0.6e6, # minimum longitude of detection region
                       'lon2': 1.3e6, # maximum longitude
                       'lat1': 1.0e6, # minimum latitude
                       'lat2': 1.3e6, # maximum latitude
                       'res': 10.,
                       'dE': 50., # maximum distance of search ellipsis from eddy center in towards the east 
                                 # (if set to 0, it will be calculated as (150. / (7. / dt)))
                       'eddy_scale_min': 0.5, # minimum factor by which eddy amplitude and area are allowed to change in one timestep
                       'eddy_scale_max': 1.5, # maximum factor by which eddy amplitude and area are allowed to change in one timestep
                       'dict': eddies, # dictionary containing detected eddies to be used when not stored in files (set to 0 otherwise)
                       'data_path': datapath, # path to the detected eddies pickle files
                       'file_root': 'test',
                       'file_spec': 'eddies_OW0.3_test',
                       'ross_path': datapath} # path to rossrad.dat containing Chelton et a1. 1998 Rossby radii

In [ ]:
# Now we track the eddies, all information needed has to be added to `tracking_parameters`
tracks = et.tracking.track(tracking_parameters, in_file=False)

We now have tracked all eddies that met the criteria specified in `tracking_parameters`. Every entry `i` in `tracks[i]` corresponds to one complete track.

In [ ]:
# The entries in `track` look like this
tracks[0]

In [ ]:
# To have a look at how the tracking performs, just pick an eddy and see whether it is tracked.
ed1 = 1
t = 0
j = 4

plt.figure(figsize=(18, 4))

for i in np.arange(0, j):
    plt.subplot(1, j, i + 1)
    plt.pcolormesh(data_int.lon, data_int.lat, data_int.OW.sel(time=tracks[ed1]['time'][i]).sel(z=100, method='nearest').values,
                   vmin=-1e-9, vmax=1e-9, cmap=cm.balance, shading='auto')
    plt.plot(tracks[ed1]['lon'][t:t + i+1], tracks[ed1]['lat'][t:t + i+1], marker='o', color='m')

In [ ]:
# We save the tracks for later use
with open(datapath
          + 'test_00610101_00611230_tracks_OW0.3'
          + '_test.pickle', 'wb') as f:
    pickle.dump(tracks, f, pickle.HIGHEST_PROTOCOL)
f.close()

In [ ]:
# This is how to open the tracks-file again (no need to do that if we just saved it)
with open(datapath
          + 'test_00610101_00611230_tracks_OW0.3'
          + '_test.pickle', 'rb') as f:
    tracks = pickle.load(f)
f.close()

### SAMPLING

Things to note for the setting of `sample_parameters`:  
1. `'start_time'` is required to be no earlier than the earliest actual date of the tracked eddies. In our case here, for the year 0002 and a 5-day temporal resolution of the data, this is `'0002-01-05'` (The `MITgcm` stores the 5-daily averages at the end of the 5-day period).
2. `'lon1'` and `'lon2'` need to be identical to `'lon1'` and `'lon2'` in `detection_parameters`.  
3. Right now, the usage of `'range'` and `'split'` has not been thouroughly tested! It seems to work for most cases though.  

`'range'`: Set to `True` if you only want to sample eddies within a certain range `'values_range'` of a property `'var_range'` in the dataset `'ds_range'`. `'var_range'` needs to be 2D (thus the `.isel(z=9)` in the example below) and interpolated to the same grid as `OW` used above. It is most likely that, if you follow this example, `'var_range'` is stored in the same dataset as `OW`. In the example below, only eddies that have a center temperature between 4 and 7 degrees C at depth level 10 (`z=9`) will be sampled and stored.  

`'split'`: Set to `True` if you want to split the sampled eddies into two categories, above and below a certain threshold value `'value_split'` of a variable `'var_split'` in the dataset `'ds_split'`. As for `'range'`, `'var_split'` needs to be 2D and interpolated to the same grid as `OW` used above. In the example below the eddies will be put into two categories: In the first category, the eddies must have a center salinity above 34.0 and in the second category, below 34.0.

In [ ]:
# For the sampling we again have to specify some parameters, defining when, where and which 
# eddies to sample.
# Because the dataset containing the sampled eddies can grow huge for larger regions and/or
# longer time periods, `eddytools.sample.sample()` writes the samples for each individual
# eddy to individual netcdf-files on disk!
sample_parameters = {'model': 'MITgcm',
                     'grid': 'cartesian',
                     'start_time': '0061-01-05', # time range start
                     'end_time': '0061-12-30', # time range end
                     'max_time': 146, # maximum length of tracks to consider
                                     # (model time steps)
                     'calendar': '360_day', # calendar, must be either 360_day or standard
                     'dt': 5, # temporal resolution of the data in days
                     'lon1': 0.6e6, # minimum longitude of detection region
                     'lon2': 1.3e6, # maximum longitude
                     'lat1': 1.0e6, # minimum latitude
                     'lat2': 1.3e6, # maximum latitude
                     'res': 10.,
                     'type': 'anticyclonic', # type of eddy
                     'lifetime': 5, # length of the eddy's track in days
                     'size': 25, # eddy size (diameter in km)
                     'range': False, # sample eddy within a range of `var_range`
                     'ds_range': data_int.isel(z=9), # dataset of `var_range`
                     'var_range': ['THETA'], # variable to base the range on
                     'value_range': [[4, 7],], # range of `var_range`
                     'split': False, # split eddies at a threshold in below and above
                     'ds_split': data_int.isel(z=9), # dataset of `var_split`
                     'var_split': ['SALT'], # variable to base split on
                     'value_split': [34.0,], # split eddies at this value
                     'sample_vars': ['THETA'], # variables to sample
                     'save_location': datapath, # where to store the netcdf files 
                     'save_name': 'test'}

The sampling can take quite long as for every eddy that fits the criteria we need to read data from disk at every time step  
The names of the files will be defined by `save_name`, the criteria you specify and the eddy number  
For this sampling parameters, the file name of the first eddy will be  
`test.anticyclonic.larger_25.longer_5.0000001.nc`  

If you set `'split'` to `True`, the file names will differ for the two categories.  
`test.anticyclonic.larger_25.longer_5.0000001.above_thr.nc` for eddies that are above `'value_split'`  
`test.anticyclonic.larger_25.longer_5.0000001.below_thr.nc`
for eddies that are below `'value_split'`.

In [ ]:
et.sample.sample(tracks, data_int, sample_parameters)

# AVERAGING

We can now average over the sampled eddies.  
As each eddy has its own file, we first need to find out how many files/samples there are, so we can loop over them and then store the datasets in a dictionary.  
Note that for large region, i.e. a lot of samples the resulting dataset could be too large to fit into memory. The `chunks={}` argument to `xr.open_dataset()` is an attempt to overcome this, however I do not know yet whether this has a lot of effect! One could also split the samples into several parts and then work on each part seperately (the number of eddies going into each average is stored, so one could later do a weighted average over the different parts!)

In [ ]:
# Find out number of samples
samples = {}
num_samples = len(glob(sample_parameters['save_location']
                       + sample_parameters['save_name'] + '.anticyclonic.larger_25.longer_5.*.nc'))

In [ ]:
# Now we load each stored sample
for i in np.arange(1, num_samples + 1):
    snum = "%07d" % (i,)
    samples[i] = xr.open_dataset(sample_parameters['save_location'] + sample_parameters['save_name'] +
                                 '.anticyclonic.larger_25.longer_5.' + str(snum) + '.nc', chunks={})

First a "preparation" is performed: Basically, all eddies are interpolated onto a normalized (in length) section crossing them zonally through the eddy center.  
For every variable specified, at every time step, the values and anomalies (with respect to surroundings) are interpolated onto the normalized section and stored according to the month the eddy was first detected. The depth profile of the surroundings is stored as well.

In [ ]:
normalized_eddies = et.average.prepare(samples, ['THETA'])

In [ ]:
normalized_eddies['THETA_anom']['12'].shape

Here for example, 3 eddies originating in December have been stored with a maximum length of 14 time steps. 55 is the length of the depth dimension, 101 the length of the normalized section (-0.5, 0.5, 0.01)

Next we can average these samples in different ways.  
Three possibilities are given with `average`  
1. seasonal -> bins the samples (of all available years) into four seasons (DJF, MAM, JJA, SON) and averages them into a seasonal climatology  
2. monthly  -> does the same for each month (so you end up with a monthly climatology)  
3. total    -> averages over all available eddies  
The three methods all return the means, standard deviations (across eddies), and the number of eddies that went into the derived quantities for a) the variables specified, b) their anomalies to the surroundings, c) the surroundings. These results are stored in the output dictionary under `output['ave']['mean'][period]['variable']`, `output['ave']['mean'][period]['variable_anom']`, `output['ave']['mean'][period]['variable_around']`, respectively. `period` refers to 1. `DJF`, `MAM`, `JJA` or `SON`, 2. `01`, `02`, `03`, etc., representing the month, or in case of the total average thie layer in the dictionary does not exist.  
Additionally, the averaged temporal evolution of the eddy centers will be stored under `output['evo'][...]`.

In [ ]:
season_eddies = et.average.seasonal(normalized_eddies, ['THETA'])

In [ ]:
plt.pcolormesh(season_eddies['ave']['mean']['DJF']['THETA_anom'], vmin=-1, vmax=1, cmap=cm.balance)
plt.colorbar()

In [ ]:
monthly_eddies = et.average.monthly(normalized_eddies, ['THETA'])

In [ ]:
plt.pcolormesh(monthly_eddies['ave']['mean']['03']['THETA_anom'], vmin=-1, vmax=1, cmap=cm.balance)
plt.colorbar()

In [ ]:
total_eddies = et.average.total(normalized_eddies, ['THETA'])

In [ ]:
plt.pcolormesh(total_eddies['ave']['mean']['THETA_anom'], vmin=-1, vmax=1, cmap=cm.balance)
plt.colorbar()